In [1]:
import io
import time
import torch
import numpy as np
import PIL
from torchvision.transforms import ToPILImage, PILToTensor
from datasets import load_dataset, Image
from piq import LPIPS, DISTS, psnr, multi_scale_ssim

class Config: pass

In [2]:
lpips_loss = LPIPS().to("cuda")
dists_loss = DISTS().to("cuda")

/home/dgj335/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dgj335/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
LSDIR = load_dataset("danjacobellis/LSDIR_val", split='validation')

In [4]:
def webp_compress_cpu(sample):
    with torch.no_grad():
        img = sample['image']
        
        t0 = time.time()
        buff = io.BytesIO()
        img.save(buff, format='WEBP', quality=0)
        webp_bytes = buff.getbuffer()
        encode_time = time.time() - t0
    
        t0 = time.time()
        Y = PIL.Image.open(buff)
        decode_time = time.time() - t0
    
        x = PILToTensor()(img).to(torch.float).unsqueeze(0)/255
        x_hat = PILToTensor()(Y).to(torch.float).unsqueeze(0)/255
        
        bpp = 8*len(webp_bytes)/(img.width*img.height)
        PSNR = psnr(x,x_hat)
        MSSIM = multi_scale_ssim(x,x_hat)
        LPIPS_dB = -10*np.log10(lpips_loss(x.to("cuda"), x_hat.to("cuda")).item())
        DISTS_dB = -10*np.log10(dists_loss(x.to("cuda"), x_hat.to("cuda")).item())   
        
    return {
        'recovered': webp_bytes,
        'cpu_encode_time': encode_time,
        'cpu_decode_time': decode_time,
        'bpp': bpp,
        'PSNR': PSNR,
        'MSSIM': MSSIM,
        'LPIPS_dB': LPIPS_dB,
        'DISTS_dB': DISTS_dB,
    }

In [5]:
device = "cpu"
cpu = LSDIR.map(webp_compress_cpu)
cpu = cpu.cast_column('recovered',Image())

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [6]:
metrics = [
     'bpp',
     'PSNR',
     'MSSIM',
     'LPIPS_dB',
     'DISTS_dB', 
     'cpu_encode_time',
     'cpu_decode_time',
]

In [7]:
for metric in metrics:
    μ = np.mean(cpu[metric])
    print(f"{metric}: {μ}")

bpp: 0.16197342237737292
PSNR: 23.65359550476074
MSSIM: 0.876294305562973
LPIPS_dB: 3.3198132446212294
DISTS_dB: 8.571886483910369
cpu_encode_time: 0.04317828464508057
cpu_decode_time: 0.00040556526184082033


In [8]:
cpu.push_to_hub("danjacobellis/LSDIR_WEBP_q0",split='validation')

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/LSDIR_WEBP_q0/commit/d1d95e90d1e58b6fbdfc45c55c55ac6985d70445', commit_message='Upload dataset', commit_description='', oid='d1d95e90d1e58b6fbdfc45c55c55ac6985d70445', pr_url=None, pr_revision=None, pr_num=None)